# Unit 7: Advanced Data Visualization
-------------------------------------

- Create subplots in `matplotlib`
- Use the `seaborn` package for more advanced plots

## 7.1. Creating subplots

In Unit 4, we created an example where we plotted a time-series data set, and it's derivative.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

plt.style.use('ggplot')

# read a .csv file into a DataFrame variable named df
fid_df = pd.read_csv('../../data/pcr-polyethylene_gc-fid.csv')
fid_df['seconds'] = fid_df['minutes'] * 60

# calculate the first derivative (change in FID signal) / (change in time)
fid_df['1st_derivative'] = fid_df['fid'].diff() / fid_df['seconds'].diff()

# plot the FID signal and its 1st derivative
# the \n in the label is the new-line character to split the label into 2 lines
fig, ax = plt.subplots(nrows=2, sharex=True)

ax[0].plot(fid_df['seconds'], fid_df['fid'])
ax[0].set_ylabel('Original\nFID Signal')

ax[1].plot(fid_df['seconds'], fid_df['1st_derivative'])
ax[1].set_xlabel('Seconds')
ax[1].set_ylabel('1st Derivative\nof FID Signal')

This can be especially useful when we have data that has multiple scales, or if we need to separate data. While it's also possible to plot on multiple y-axes, this can be more difficult to interpret.

--------------
## Next Steps:

1. Complete the [Unit 7 Problems](./unit07-solutions.ipynb) to test your understanding
2. Advance to [Unit 8](../08-image-analysis/unit08-lesson.ipynb) when you're ready for the next step